# **Crawl Data https://www.flickr.com**

## Install necessary libraries

In [1]:
!apt-get update
!apt-get install -y wget
%pip install tqdm
%pip install selenium
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,861 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,446 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,167 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,132 kB]
Get:14 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packa

## Import libraries

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup  # For parsing HTML content
from urllib.parse import urljoin, urlparse  # For handling URLs
import urllib.request  # For making HTTP requests
import time  # For handling time-related operations
import os  # For interacting with the operating system (relate to dir, folder, file)
from tqdm import tqdm  # For displaying progress bars (visualize progress)
import concurrent.futures  # For multi-threading
import json  # For writing to a text file
from PIL import Image  # For handling images

## Crawl image links from website (Brute force)

In [3]:
URL = "https://www.flickr.com/search/?text="
search_term = 'cat'

In [4]:
def get_url_images(driver, max_images):
    """
    Crawl the urls of images by term

    Parameters:
    term (str): The name of animal, plant, scenery, furniture

    Returns:
    urls (list): List of urls of images
    """
    urls = [] # Store links of all images we want
    more_content_available = True # Checking variable for manage lazy loading page

    # Crawl url images until the number of link = max img we want or the page don't have any image
    while len(urls) < max_images and more_content_available:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        img_tags = soup.find_all("img")

        # Extract url image from HTML source
        for img in img_tags:
            if len(urls) >= max_images:
                break
            if 'src' in img.attrs:
                href = img.attrs['src']
                img_path = urljoin(url, href)
                img_path = img_path.replace("_m.jpg", "_b.jpg").replace("_n.jpg", "_b.jpg").replace("_w.jpg", "_b.jpg")
                if img_path == "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
                    continue
                urls.append(img_path)


        # Click load more button or scroll page for more image
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[@id="yui_3_16_0_1_1721642285931_28620"]'))
            )
            load_more_button.click()
            time.sleep(2) # Wait for generating content
        except:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) # Wait for generating content

            # Check number of new generating image
            new_soup = BeautifulSoup(driver.page_source, "html.parser")
            new_img_tags = new_soup.find_all("img", loading_="lazy")
            if len(new_img_tags) == len(img_tags):
                more_content_available = False
            img_tags = new_img_tags

    return urls

In [5]:
url = URL + search_term
max_images = 200

In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
driver.get(url)

urls = get_url_images(driver, max_images)
print(f"Number of images retrieved: {len(urls)}")
driver.quit()

Number of images retrieved: 200


In [7]:
urls

['https://live.staticflickr.com/5598/14934282524_344c84246b_b.jpg',
 'https://live.staticflickr.com/4838/45925416992_c9caac8cb9_b.jpg',
 'https://live.staticflickr.com/7697/17026317426_bb3acf19fb_b.jpg',
 'https://live.staticflickr.com/8750/16386660144_a6c4026657_b.jpg',
 'https://live.staticflickr.com/2813/32711702733_7cdd9dbb3a_b.jpg',
 'https://live.staticflickr.com/4728/39256672581_c48569e3a2_b.jpg',
 'https://live.staticflickr.com/7073/7190755946_ea97e85765_b.jpg',
 'https://live.staticflickr.com/3946/14934292534_32b0302f2d_b.jpg',
 'https://live.staticflickr.com/5036/5881933297_7974eaff82_b.jpg',
 'https://live.staticflickr.com/4066/4638426067_1a979d3690_b.jpg',
 'https://live.staticflickr.com/2602/3977203168_b9d02a0233_b.jpg',
 'https://live.staticflickr.com/3940/15504684310_f555c88915_b.jpg',
 'https://live.staticflickr.com/7313/9775005856_9b5e0ebe16_b.jpg',
 'https://live.staticflickr.com/1729/41676479745_ae6d27ee9d_b.jpg',
 'https://live.staticflickr.com/1701/24811748270_3102

## Crawl image links from website (More reusable and efficient)

In [8]:
class UrlScraper:
    # Constructor
    def __init__(self, url_template, max_images=50, max_workers=4):
        self.url_template = url_template # Link crawl
        self.max_images = max_images # Max images
        self.max_workers = max_workers # Thread
        self.setup_environment() # Call for set up environment

    # Set up environment for selenium
    def setup_environment(self):
        os.environ['PATH'] += ':/usr/lib/chromium-browser/'
        os.environ['PATH'] += ':/usr/lib/chromium-browser/chromedriver/'

    def get_url_images(self, term):
        """
        Crawl the urls of images by term

        Parameters:
        term (str): The name of animal, plant, scenery, furniture

        Returns:
        urls (list): List of urls of images
        """

        # Initialize Chrome driver
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)

        url = self.url_template.format(search_term=term)
        driver.get(url)

        # Start crawl urls of image like brute force - the same mechanism with this but add some feature
        urls = []
        more_content_available = True

        pbar = tqdm(total=self.max_images, desc=f"Fetching images for {term}") # Set up for visualize progress

        while len(urls) < self.max_images and more_content_available:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            img_tags = soup.find_all("img")

            for img in img_tags:
                if len(urls) >= self.max_images:
                    break
                if 'src' in img.attrs:
                    href = img.attrs['src']
                    img_path = urljoin(url, href)
                    img_path = img_path.replace("_m.jpg", "_b.jpg").replace("_n.jpg", "_b.jpg").replace("_w.jpg", "_b.jpg")
                    if img_path == "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
                        continue
                    urls.append(img_path)
                    pbar.update(1)

            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@id="yui_3_16_0_1_1721642285931_28620"]'))
                )
                load_more_button.click()
                time.sleep(2)
            except:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

                new_soup = BeautifulSoup(driver.page_source, "html.parser")
                new_img_tags = new_soup.find_all("img", loading_="lazy")
                if len(new_img_tags) == len(img_tags):
                    more_content_available = False
                img_tags = new_img_tags

        pbar.close()
        driver.quit()
        return urls

    def scrape_urls(self, categories):
        """
        Call get_url_images method to get all urls of any object in categories\

        Parameter:
        categories (dictionary): the dict of all object we need to collect image with format categories{"name_object": [value1, value2, ...]}

        Returns:
        all_urls (dictionary): Dictionary of urls of images
        """
        all_urls = {category: {} for category in categories}

        # Handle multi-threading for efficent installation
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            future_to_term = {executor.submit(self.get_url_images, term): (category, term)
                              for category, terms in categories.items() for term in terms}

            for future in tqdm(concurrent.futures.as_completed(future_to_term), total=len(future_to_term), desc="Overall Progress"):
                category, term = future_to_term[future]
                try:
                    urls = future.result()
                    all_urls[category][term] = urls
                    print(f"\nNumber of images retrieved for {term}: {len(urls)}")
                except Exception as exc:
                    print(f"\n{term} generated an exception: {exc}")
        return all_urls

    def save_to_file(self, data, filename):
        """
        Save the data to a JSON file.

        Parameters:
        data (dict): The data to be saved.
        filename (str): The name of the JSON file.

        Returns:
        None
        """
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Data saved to {filename}")

In [9]:
categories = {
    "animal": ["Monkey", "Elephant", "cows", "Cat", "Dog", "bear", "fox", "Civet", "Pangolins", "Rabbit", "Bats", "Whale", "Cock", "Owl", "flamingo", "Lizard", "Turtle", "Snake", "Frog", "Fish", "shrimp", "Crab", "Snail", "Coral", "Jellyfish", "Butterfly", "Flies", "Mosquito", "Ants", "Cockroaches", "Spider", "scorpion", "tiger", "bird", "horse", "pig", "Alligator", "Alpaca", "Anteater", "donkey", "Bee", "Buffalo", "Camel", "Caterpillar", "Cheetah", "Chicken", "Dragonfly", "Duck", "panda", "Giraffe"],
    "plant": ["Bamboo", "Apple", "Apricot", "Banana", "Bean", "Wildflower", "Flower", "Mushroom", "Weed", "Fern", "Reed", "Shrub", "Moss", "Grass", "Palmtree", "Corn", "Tulip", "Rose", "Clove", "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani", "Lantana", "Hibiscus", "Bougainvillea", "Pea", "OrchidTree", "RangoonCreeper", "Jackfruit", "Cottonplant", "Corneliantree", "Coffeeplant", "Coconut", "wheat", "watermelon", "radish", "carrot"],
    "furniture": ["bed", "cabinet", "chair", "chests", "clock", "desks", "table", "Piano", "Bookcase", "Umbrella", "Clothes", "cart", "sofa", "ball", "spoon", "Bowl", "fridge", "pan", "book"],
    "scenery": ["Cliff", "Bay", "Coast", "Mountains", "Forests", "Waterbodies", "Lake", "desert", "farmland", "river", "hedges", "plain", "sky", "cave", "cloud", "flowergarden", "glacier", "grassland", "horizon", "lighthouse", "plateau", "savannah", "valley", "volcano", "waterfall"]
}

In [9]:
# Test

categories = {
    "animal": ["horse", "pig"],
    "plant": ["apple", "carrot", "flower"],
    "furniture": ["table", "Piano", "Bookcase"],
    "scenery": ["sky", "cave", "cloud"]
}

In [10]:
urltopic = {"flickr": "https://www.flickr.com/search/?text={search_term}"}
scraper = UrlScraper(url_template=urltopic["flickr"], max_images=20, max_workers=5)
image_urls = scraper.scrape_urls(categories)

Fetching images for apple:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for apple:   5%|▌         | 1/20 [00:00<00:02,  7.07it/s]


Fetching images for carrot:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for flower:   5%|▌         | 1/20 [00:00<00:03,  6.31it/s]



Fetching images for horse:   0%|          | 0/20 [00:00<?, ?it/s]




Fetching images for pig:   0%|          | 0/20 [00:00<?, ?it/s]


Fetching images for carrot:   5%|▌         | 1/20 [00:00<00:05,  3.56it/s]


Fetching images for carrot:  10%|█         | 2/20 [00:00<00:03,  5.00it/s]




Fetching images for pig:   5%|▌         | 1/20 [00:00<00:05,  3.44it/s]



Overall Progress:  27%|██▋       | 3/11 [00:16<00:34,  4.30s/it]


Number of images retrieved for apple: 20

Number of images retrieved for flower: 20

Number of images retrieved for pig: 20

Number of images retrieved for carrot: 20

Number of images retrieved for horse: 20



Fetching images for Piano:   5%|▌         | 1/20 [00:00<00:04,  4.37it/s]

Fetching images for table:   0%|          | 0/20 [00:00<?, ?it/s]


Fetching images for Bookcase:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for table:   5%|▌         | 1/20 [00:00<00:03,  5.58it/s]



Fetching images for cave:   0%|          | 0/20 [00:00<?, ?it/s]


Fetching images for Bookcase:   5%|▌         | 1/20 [00:00<00:03,  5.63it/s]



Fetching images for cave:   5%|▌         | 1/20 [00:00<00:04,  4.21it/s]




Fetching images for sky:   0%|          | 0/20 [00:00<?, ?it/s]




Overall Progress:  55%|█████▍    | 6/11 [00:30<00:23,  4.60s/it]


Number of images retrieved for Piano: 20


Fetching images for cave: 100%|██████████| 20/20 [00:12<00:00,  1.55it/s]



Number of images retrieved for table: 20


Overall Progress:  91%|█████████ | 10/11 [00:31<00:01,  1.71s/it]


Number of images retrieved for Bookcase: 20

Number of images retrieved for cave: 20

Number of images retrieved for sky: 20



Overall Progress: 100%|██████████| 11/11 [00:45<00:00,  4.13s/it]


Number of images retrieved for cloud: 20


In [11]:
scraper.save_to_file(image_urls, 'image_urls.json')

Data saved to image_urls.json


## Downloading Images from JSON file

In [12]:
class ImageDownloader:
    def __init__(self, json_file, download_dir='Dataset', max_workers=4, delay=1):
        self.json_file = json_file  # file containing URLs of images in JSON format
        self.download_dir = download_dir  # Folder name for storing images
        self.max_workers = max_workers  # Number of threads
        self.delay = delay  # Polite delay: when we send request too much to the server for downloading images without polite delay, it will crash or prevent your IP
        self.filename = set()  # To store filename directories
        self.setup_directory()  # Set up the folder structure

    def setup_directory(self):
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)

    def read_json(self):
        """
        Read the JSON file and return the data.

        Returns:
        data (dict): The data read from the JSON file.
        """
        with open(self.json_file, 'r') as file:
            data = json.load(file)
        return data

    def is_valid_url(self, url):
        """
        Check if the URL is valid.

        Parameters:
        url (str): The URL to be checked.

        Returns:
        bool: True if the URL is valid, False otherwise.
        """
        try:
            with urllib.request.urlopen(url) as response:
                if response.status == 200 and 'image' in response.info().get_content_type():
                    return True
        except Exception:
            return False

    def download_image(self, url, category, term, pbar):
        """
        Download the image from the given URL.

        Parameters:
        url (str): The URL of the image to be downloaded.
        category (str): The category of the image.
        term (str): The term or keyword associated with the image.
        pbar (tqdm): The progress bar object.

        Returns:
        str: A message indicating the status of the download.
        """
        if not self.is_valid_url(url):
            pbar.update(1)
            return f"Invalid URL: {url}"

        category_dir = os.path.join(self.download_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

        term_dir = os.path.join(category_dir, term)
        if not os.path.exists(term_dir):
            os.makedirs(term_dir)

        filename = os.path.join(term_dir, os.path.basename(urlparse(url).path))

        self.filename.add(filename)  # Record the filename directory

        try:
            urllib.request.urlretrieve(url, filename)
            pbar.update(1)
            return f"Downloaded: {url}"
        except Exception as e:
            pbar.update(1)
            return f"Failed to download {url}: {str(e)}"

    def download_images(self):
        """
        Download images from the JSON file.

        Returns:
        None
        """
        data = self.read_json()
        download_tasks = []

        total_images = sum(len(urls) for terms in data.values() for urls in terms.values())
        with tqdm(total=total_images, desc="Downloading images") as pbar:
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                for category, terms in data.items():
                    for term, urls in terms.items():
                        for url in urls:
                            download_tasks.append(executor.submit(self.download_image, url, category, term, pbar))
                            time.sleep(self.delay)  # Polite delay

                for future in concurrent.futures.as_completed(download_tasks):
                    print(future.result())

        self.export_filename()

    def export_filename(self):
        """
        Export the filename directories to a text file.

        Returns:
        None
        """
        with open('filename.txt', 'w') as file:
            for filename in sorted(self.filename):
                file.write(f"{filename}\n")


In [13]:
downloader = ImageDownloader(json_file='image_urls.json', download_dir='Dataset', max_workers=4, delay=1)
downloader.download_images()

Downloaded: https://live.staticflickr.com/3118/2901804012_005e0f78bd_b.jpg
Downloaded: https://live.staticflickr.com/485/31643806045_8bc983e054_b.jpg
Downloaded: https://live.staticflickr.com/4505/24365462008_a39091c56b_b.jpg
Downloaded: https://live.staticflickr.com/4442/36356331310_4a9b2de296_b.jpg
Downloaded: https://live.staticflickr.com/1423/709716481_7c5029de9c_b.jpg
Downloaded: https://live.staticflickr.com/647/21982802134_e4c135c7f8_b.jpg
Downloaded: https://live.staticflickr.com/8454/8032857022_95525a810c_b.jpg
Downloaded: https://live.staticflickr.com/5256/5499887991_15b646080b_b.jpg
Downloaded: https://live.staticflickr.com/6165/6236665025_d14709a7bf_b.jpg
Downloaded: https://live.staticflickr.com/6054/6276863304_4b9f30551f_b.jpg
Downloaded: https://live.staticflickr.com/3685/9757721741_a8d675e9b7_b.jpg
Downloaded: https://live.staticflickr.com/2345/2133721692_6df8dcab88_b.jpg
Downloaded: https://live.staticflickr.com/2101/5718273522_b35316446f_b.jpg
Downloaded: https://live

In [14]:
downloader.export_filename()

## Processing Dataset

In [15]:
def check_and_preprocess_images(image_dir):
    """
    Check and preprocess images in the specified directory.

    Parameters:
    image_dir (str): The directory containing the images to be checked and preprocessed.

    Returns:
    None
    """
    for root, _, files in os.walk(image_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    # Check if image is smaller than 50x50 pixels
                    if img.size[0] < 50 or img.size[1] < 50:
                        os.remove(file_path)
                        print(f"Deleted {file_path}: Image too small ({img.size[0]}x{img.size[1]})")
                        continue

                    # Convert non-RGB images to RGB
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        img.save(file_path)
                        print(f"Converted {file_path} to RGB")

            except Exception as e:
                # If file is not an image, delete it
                os.remove(file_path)
                print(f"Deleted {file_path}: Not an image or corrupted file ({str(e)})")

In [16]:
check_and_preprocess_images('Dataset')

Zip folder dataset

In [17]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [18]:
!zip -r /content/drive/MyDrive/Clean_Dataset.zip Dataset

  adding: Dataset/ (stored 0%)
  adding: Dataset/plant/ (stored 0%)
  adding: Dataset/plant/carrot/ (stored 0%)
  adding: Dataset/plant/carrot/23819862783_570fdc38f3_b.jpg (deflated 17%)
  adding: Dataset/plant/carrot/51660904952_0f95dba27a_b.jpg (deflated 0%)
  adding: Dataset/plant/carrot/22692217656_346870a506_b.jpg (deflated 0%)
  adding: Dataset/plant/carrot/3024436378_02e824ce7f_b.jpg (deflated 0%)
  adding: Dataset/plant/carrot/4818307176_b16243959e_b.jpg (deflated 4%)
  adding: Dataset/plant/carrot/6400529427_051113dc23_b.jpg (deflated 0%)
  adding: Dataset/plant/carrot/6237042125_1fcd6c7def_b.jpg (deflated 0%)
  adding: Dataset/plant/carrot/50934286617_3eddf3835f_b.jpg (deflated 0%)
  adding: Dataset/plant/carrot/5451540472_27ac10b9ff_b.jpg (deflated 0%)
  adding: Dataset/plant/carrot/51037125227_998fb64e90_b.jpg (deflated 3%)
  adding: Dataset/plant/carrot/3058918828_927539b9c1_b.jpg (deflated 0%)
  adding: Dataset/plant/carrot/764833075_71bdba6ae0_b.jpg (deflated 0%)
  addin

In [19]:
!cp filename.txt /content/drive/MyDrive/filename.txt